# ICLR 2025 会议论文：深度主题分析与可视化

本 Notebook 的目标是对 `main.py` 流程生成的 `...with_topics.csv` 文件进行可视化,这里以ICLR 2025为示例。

主要探索以下几个核心问题：
1.  **热门主题**：哪些研究方向的投稿量最大？
2.  **质量与认可度**：哪些主题的论文平均分数最高？
3.  **最终接收情况**：各个主题的最终接收率（Oral, Spotlight, Poster）分布如何？

---
**第一步：导入所有需要的工具库并设置绘图样式。**

In [ ]:
import pandas as pd
import numpy as np
import yaml

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['axes.unicode_minus'] = False 

---
**第二步：加载数据。**

加载 `data/processed/` 目录下的 CSV 文件。这个文件包含了论文信息、审稿结果以及通过 `BERTopic` 计算出的主题ID。

In [ ]:
file_path = '../data/processed/ICLRcc_2025_Conference_papers_reviews_with_topics.csv'

try:
    df = pd.read_csv(file_path)
    print("✅ 数据加载成功!")
    print(f"共加载 {len(df)} 篇论文。")
except FileNotFoundError:
    print(f"❌ 错误: 文件 '{file_path}' 未找到。")
    print("请确认文件名是否正确，以及你是否已经成功运行了主题建模脚本。")
    df = pd.DataFrame()

---
**第三步：数据预处理与核心指标计算。**

在进行可视化之前，需要做一些数据预处理操作：
1.  **移除离群点**：`BERTopic` 会将不属于任何主题（离群点）的论文标记为 `-1`。
2.  **标准化决策**：将 `decision` 列中多样化的文本（如 "Accept (Poster)"）统一为标准分类（如 "Poster"）。
3.  **计算统计量**：按主题（`Topic`）分组，计算每个主题的论文总数、平均分、各类决策的比例以及总接收率。

完成这一步后，将得到一个名为 `analysis_df` 的核心数据表，后续所有图表都将基于它来绘制。

**这段代码里给出了各个Bertopic主题分类概括后的名称，即topic_labels字典**

In [ ]:
# --- 3. 数据预处理、统计计算与主题名称映射  ---

if 'df' in locals() and isinstance(df, pd.DataFrame) and not df.empty:
    
    # 3.1 移除离群点
    df_filtered = df[df['Topic'] != -1].copy()
    print(f"移除了 {len(df) - len(df_filtered)} 篇离群论文。剩余 {len(df_filtered)} 篇用于分析。")

    # 3.2 标准化 'decision' 列
    def clean_decision(decision_str):
        decision_str = str(decision_str).lower()
        if 'oral' in decision_str: return 'Oral'
        if 'spotlight' in decision_str: return 'Spotlight'
        if 'poster' in decision_str: return 'Poster'
        if 'reject' in decision_str: return 'Reject'
        return 'N/A'
    df_filtered['clean_decision'] = df_filtered['decision'].apply(clean_decision)

    # 3.3 计算基础统计
    topic_stats = df_filtered.groupby('Topic').agg(
        paper_count=('id', 'count'),
        avg_rating=('avg_rating', 'mean')
    ).reset_index()

    # 3.4 统计决策数量
    decision_counts = df_filtered.groupby(['Topic', 'clean_decision'])['id'].count().unstack(fill_value=0)

    # 3.5 合并数据与计算接收率
    analysis_df = pd.merge(topic_stats, decision_counts, on='Topic', how='left').fillna(0)
    decision_types = ['Oral', 'Spotlight', 'Poster', 'Reject', 'N/A']
    for dtype in decision_types:
        if dtype not in analysis_df.columns:
            analysis_df[dtype] = 0
    accepted_papers = analysis_df['Oral'] + analysis_df['Spotlight'] + analysis_df['Poster']
    total_papers_in_scope = accepted_papers + analysis_df['Reject'] + analysis_df['N/A']
    analysis_df['acceptance_rate'] = (accepted_papers / total_papers_in_scope).fillna(0)
    
    # --- 3.6 添加可读的主题名称 ---
    labels_path = '../results/iclr_2025_analysis/topic_labels.yaml' 
    topic_labels = None 
    try:
        # 尝试从外部文件加载
        with open(labels_path, 'r', encoding='utf-8') as f:
            topic_labels = yaml.safe_load(f)
        
        # 检查加载结果是否有效 (例如，不是空的或None)
        if not isinstance(topic_labels, dict):
            print(f"⚠️ 解析主题名称文件 '{labels_path}' 的内容似乎不是有效的字典。将使用默认名称。")
            topic_labels = None # 标记为加载失败
            raise FileNotFoundError # 触发 except 块的备用逻辑

        print(f"✅ 成功从外部文件 '{labels_path}' 加载主题名称。")
        
        # --- 成功路径：使用加载的字典进行映射，并正确处理fillna ---
        analysis_df['Topic_Name'] = analysis_df['Topic'].map(topic_labels)
        # 使用赋值操作修正 FutureWarning
        fallback_names = analysis_df['Topic'].apply(lambda x: f"Topic {x}")
        analysis_df['Topic_Name'] = analysis_df['Topic_Name'].fillna(fallback_names)

    except (FileNotFoundError, ImportError, NameError) as e: 
         # --- 失败路径：处理文件未找到、yaml库问题或加载失败 ---
         if isinstance(e, FileNotFoundError):
             print(f"⚠️ 未找到主题名称文件: {labels_path}。将使用默认的 'Topic [ID]' 作为名称。")
         elif isinstance(e, (ImportError, NameError)): # NameError for yaml if not imported
             print(f"⚠️ PyYAML 库未安装或导入失败。请确保已导入 `yaml`。将使用默认名称。")
         # (topic_labels is None 的情况已在 try 块内部处理)

         analysis_df['Topic_Name'] = analysis_df['Topic'].apply(lambda x: f"Topic {x}")

    # 3.7 整理列的顺序
    final_columns = [
        'Topic', 'Topic_Name', 'paper_count', 'avg_rating', 'acceptance_rate', 
        'Oral', 'Spotlight', 'Poster', 'Reject', 'N/A'
    ]
    final_columns = [col for col in final_columns if col in analysis_df.columns]
    analysis_df = analysis_df[final_columns]
    
    print("✅ 数据预处理与统计计算完成！")

    # 3.8 显示最终处理好的数据表的前几行
    display(analysis_df.head())

else:
    print("⚠️ 'df' DataFrame 不存在或为空。请先加载数据。")
    analysis_df = pd.DataFrame() 

---
**第四步：可视化分析。**

#### **图表一：哪个方向最“热”**
按论文数量从多到少排序。

In [ ]:
# --- 4.1 可视化：论文数量  ---
top_n = 65  
top_topics_by_count = analysis_df.sort_values(by='paper_count', ascending=False).head(top_n)

HEIGHT_PER_ITEM = 0.4   
BASE_HEIGHT = 3.0       
MIN_HEIGHT = 8.0       

num_items = len(top_topics_by_count)
dynamic_height = (num_items * HEIGHT_PER_ITEM) + BASE_HEIGHT
figure_height = max(MIN_HEIGHT, dynamic_height)

plt.figure(figsize=(14, figure_height))

sns.barplot(x='paper_count', y='Topic_Name', data=top_topics_by_count, orient='h', palette='viridis', hue='Topic_Name', legend=False)
plt.title(f'Top {top_n} Topics by Paper Count at ICLR 2025', fontsize=20)
plt.xlabel('Number of Papers', fontsize=14)
plt.ylabel('Topic Name (sorted by paper count)', fontsize=14)

if num_items > 30:
    plt.yticks(fontsize=10)
else:
    plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()

#### **图表二：哪个方向最受审稿人青睐？ (各个主题平均分)**

In [ ]:
# --- 4.2 可视化：平均审稿分数  ---
top_n = 65 # 主题数量

# --- 按 'avg_rating' 排序 ---
top_topics_by_rating = analysis_df.sort_values(by='avg_rating', ascending=False).head(top_n)

# --- 动态计算图表高度 ---
HEIGHT_PER_ITEM = 0.4
BASE_HEIGHT = 3.0
MIN_HEIGHT = 8.0

num_items = len(top_topics_by_rating)
dynamic_height = (num_items * HEIGHT_PER_ITEM) + BASE_HEIGHT
figure_height = max(MIN_HEIGHT, dynamic_height)

plt.figure(figsize=(14, figure_height))
# ---

sns.barplot(x='avg_rating', y='Topic_Name', data=top_topics_by_rating, orient='h', palette='plasma', hue='Topic_Name', legend=False)

plt.title(f'Top {top_n} Topics by Average Rating', fontsize=20)
plt.xlabel('Average Rating', fontsize=14)
plt.ylabel('Topic Name', fontsize=14) 

if num_items > 30:
    plt.yticks(fontsize=10)
else:
    plt.yticks(fontsize=12)

plt.xlim(left=analysis_df['avg_rating'].min() * 0.9 if not analysis_df['avg_rating'].empty else 0)
plt.tight_layout()
plt.show()

#### **图表三：各个主题的分布情况？ (按接收率排序)**

In [ ]:
# --- 4.3 可视化：标准化的决策构成比例，并标注总数 ---
import matplotlib.ticker as mtick

top_n = 65 
top_topics_df = analysis_df.sort_values(by='acceptance_rate', ascending=False).head(top_n)

# --- 1. 准备绘图数据并进行标准化 ---
decision_columns_with_na = ['Oral', 'Spotlight', 'Poster', 'Reject', 'N/A']
plot_data = top_topics_df.set_index('Topic_Name')[decision_columns_with_na]
plot_data_normalized = plot_data.div(plot_data.sum(axis=1), axis=0)

# --- 2. 动态计算图表高度 ---
HEIGHT_PER_ITEM = 0.5   
BASE_HEIGHT = 4.0
MIN_HEIGHT = 10.0       

num_items = len(plot_data_normalized)
dynamic_height = (num_items * HEIGHT_PER_ITEM) + BASE_HEIGHT
figure_height = max(MIN_HEIGHT, dynamic_height)

# --- 3. 绘图 ---
plt.rcParams['figure.figsize'] = (18, figure_height) 

ax = plot_data_normalized.plot( 
    kind='barh', 
    stacked=True, 
    colormap='viridis',
    width=0.8 
) 

# --- 4. 在每个条形图右侧标注总论文数 ---
count_map = top_topics_df.set_index('Topic_Name')['paper_count']

for i, topic_name in enumerate(plot_data_normalized.index):
    total_papers = count_map[topic_name]
    
    ax.text(
        1.01, 
        i,    
        f'n={total_papers}', 
        va='center',
        fontsize=10,
        color='black',
        fontweight='bold'
    )

plt.title(f'Top {top_n} Topics: Decision Breakdown at ICLR 2025', fontsize=20, pad=30) 
plt.xlabel('Proportion of Papers within Each Topic', fontsize=14)
plt.ylabel('Topic Name (Sorted by Acceptance Rate)', fontsize=14)

plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.xlim(0, 1) 

if num_items > 30: 
    plt.yticks(fontsize=10) 
else: 
    plt.yticks(fontsize=12) 

plt.gca().invert_yaxis() 

plt.legend(
    title='Decision Type', 
    loc='upper center', 
    bbox_to_anchor=(0.5, 1.04), 
    ncol=5, 
    frameon=False, 
    fontsize=12
) 

plt.tight_layout()
plt.show() 

plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']

---
**第五步：查看最终的统计总表。**


In [ ]:
# --- 5. 最终统计表 ---


display_cols = [
    'Topic_Name', 
    'paper_count', 
    'avg_rating', 
    'acceptance_rate', 
    'Oral', 
    'Spotlight', 
    'N/A'  
]

final_table = analysis_df[display_cols].sort_values(by='acceptance_rate', ascending=False).head(top_n).copy()

final_table.style.format({
    'avg_rating': '{:.2f}',           # 平均分保留两位小数
    'acceptance_rate': '{:.2%}',      # 接收率格式化为百分比
    'Oral': '{:d}',                   # Oral 论文数显示为整数
    'Spotlight': '{:d}',              # Spotlight 论文数显示为整数
    'Poster': '{:d}',                 # Poster 论文数显示为整数
    'Reject': '{:d}',                 # Reject 论文数显示为整数
    'N/A': '{:d}'                     # N/A (撤稿) 论文数显示为整数
}).bar(subset=['paper_count'], color='#5fba7d', align='left') \
  .bar(subset=['avg_rating'], color='#d65f5f', align='left', vmin=analysis_df['avg_rating'].min()) \
  .set_caption(f"Top {top_n} Topics Analysis Summary (Sorted by Acceptance Rate)") 